In [79]:
import selenium
import requests as rq
import os
import bs4 #객체 읽기
import urllib.parse as up #한국어를 url로 바꾸기
import mysql.connector as sqlcnt #mysql 연결 -> 데이터 베이스
import datetime #검색한 날짜 입력하기 위해
import pytz # 서울 표준시로 바꾸기 위함

In [3]:
# 쿠팡 url
item,page=None,None
url=f'https://www.coupang.com/np/search?q={item}&channel=user&page={page}'
header={'Refer':""
        ,"User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'
        ,"Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"} # 이부분이 쿠팡은 추가로 필요함

In [2]:
#검색어를 url 형식에 맞게 encoding해줘야함
def encoder(x:str):
   return up.quote(x.encode('utf-8')) #우선 utf-8로 되어있어야함

def urlmkr(item,page):
   item=encoder(item)
   return f'https://www.coupang.com/np/search?q={item}&channel=user&page={page}'

In [8]:
# 1단계 : 객체 생성
# coupang_search=rq.Session()
# coupang_search.headers.update(header)

In [95]:
# 2단계  : url 수정 후 request
# 3단계 : 위의 틀을 기반으로 검색어를 자유롭게 바꿀수 있게끔 함수화 하기
def coupang(item):
    #해더
    header={'Refer':""
        ,"User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'
        ,"Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"}
    def encoder(item:str):
       return up.quote(item.encode('utf-8'))
    def urlmkr(item,page):
        item=encoder(item)
        return f'https://www.coupang.com/np/search?q={item}&channel=user&page={page}'
    
    #객체 생성
    coupang_search=rq.Session()
    coupang_search.headers.update(header)
    
    #검색시작
    page=0 #시작 페이지 1부터
    items=[]
    empty=[]
    while True:
        page+=1
        url=urlmkr(item,page)
        try:
            search=coupang_search.get(url,timeout=5)
        # 검색 중지 조건 1. 모두 검색 완료 / 2. 페이지 오류-> 검색시간이 5초를 초과
            if search.status_code!=200:
                print(page)
                break
        except:
            break
        html=bs4.BeautifulSoup(search.text)
        # 판매목록 세부화
        items_html=html.select("li.search-product") #상품 하나씩 조회
        add_adress='www.coupang.com' #상품주소 에 추가해야 이동가능
        for sub_html in items_html:
            href=sub_html.select("a")[0].get_attribute_list("href")[0] #상품주소
            
            name=sub_html.select('div.name')[0].text.strip()   #상품명

            if sub_html.select("em.rating"): # 평점이 없을수도 있음
                rating=sub_html.select("em.rating")[0].text.strip() #상품 평점

            if sub_html.select('span.rating-total-count'): #리뷰가 없을수도 있음
                count_review=sub_html.select('span.rating-total-count')[0].text.strip() #상품 리뷰수

            price=sub_html.select('strong.price-value')[0].text.strip() #상품가격
            
            # 검색한 시간을 기록하기 위해선(날짜가 변경돼 차후 다시 검색했을시 없을 수도 있으므로) 아래의 코드가 필요
            enc=pytz.timezone("Asia/Seoul")
            enc_time=datetime.datetime.now(enc) # 서울 표준시로 변경

            description={'item_adress':add_adress+href,
                        'name':name,
                        'rating':rating,
                        'count-_review':count_review,
                        'price':price,
                        "search_time":f'{enc_time.year}-{enc_time.month}-{enc_time.day}'}
            items.append(description)
    return {"item":item,'result':items,'false':empty}

In [96]:
# 테스트
test=coupang("코카콜라")


In [97]:
test["result"][0]

{'item_adress': 'www.coupang.com/vp/products/7698998310?itemId=20608085096&vendorItemId=86170602196&sourceType=srp_product_ads&clickEventId=f176fec0-7ed7-11ee-9ac4-223cd49cc085&korePlacement=15&koreSubPlacement=1&clickEventId=f176fec0-7ed7-11ee-9ac4-223cd49cc085&korePlacement=15&koreSubPlacement=1',
 'name': '오리지널 코카콜라 190ml 1박스 30캔입 미니캔 캔콜라, 30개',
 'rating': '4.5',
 'count-_review': '(113)',
 'price': '24,000',
 'search_time': '2023-11-9'}